## Diagonalization

In this section we describe a basic computational method that can be used to approximate eigenvalues.

- Demonstrate SciPy Method
- Explain basis for Power Method
- Show examples
- Power Method with Shifts?

Since the action of an $n\times n$ matrix $A$ on its eigenvectors is easy to understand, we might try to understand the action of a matrix on an arbitrary vector $X$ by writing $X$ as a linear combination of eigenvectors.  This will always be possible if the eigenvectors *form a basis for* $\mathbb{R}^n$.  Suppose $\{V_1, V_2, ..., V_n\}$ are the eigenvectors of $A$ and they do form a basis for $\mathbb{R}^n$.  Then for any vector $X$, we can write 
$X = c_1V_1 + c_2V_2 + ... c_nV_n$.  The product $AX$ can easily be computed then by multiplying each eigenvector component by the corresponding eigenvalue.

$$
\begin{eqnarray*}
AX & = & A(c_1V_1 + c_2V_2 + ... c_nV_n) \\
   & = & c_1AV_1 + c_2AV_2 + ... c_nAV_n \\
   & = & c_1\lambda_1V_1 + c_2\lambda_2V_2 + ... c_n\lambda_nV_n
\end{eqnarray*}
$$



### Example 1:  Shear

Consider a matrix that represents a shear along the line $x_1=x_2$.  Demonstrate the action of the matrix as the sum of actions on the eigenvector components.

### Computation with SciPy

We now demonstrate how to compute eigenvalues and eigenvectors for any square matrix using the function $\texttt{eig}$ from the $\texttt{linalg}$ module of the SciPy library.  This function accepts an $n\times n$ array representing a matrix and returns two arrays, one containing the eigenvalues, the other the eigenvectors.  We examine the usage by supplying our projection matrix as the argument. 

In [14]:
import numpy as np
import laguide as lag
import scipy.linalg as SLA

A = np.array([[0.2, -0.4],[-0.4, 0.8]])
print(A)
print('\n')

evalues,evectors = SLA.eig(A)

print(evalues)

[[ 0.2 -0.4]
 [-0.4  0.8]]


[0.+0.j 1.+0.j]


The array of eigenvalues contains two entries that are in the format of $\alpha + \beta j$, which represents a complex number.  The symbol $j$ is used for the imaginary unit.  The value of $\beta$ is zero for both of the eigenvalues, which means that they are both real numbers.  The results confirm our conclusions that the eigenvalues are 0 and 1.

Next let's look at the array of eigenvectors.  We can slice the array into columns to give us convenient access to the vectors.

In [15]:
print(evectors)

V_1 = evectors[:,0:1]
V_2 = evectors[:,1:2]


[[-0.89442719  0.4472136 ]
 [-0.4472136  -0.89442719]]


We may not recognize the eigenvectors as those we found previously, but recall that eigenvectors are not unique.  The $\texttt{eig}$ function scales all the eigenvectors to unit length, and we arrive at the same result if we scale our choice of eigenvector.

In [18]:
V = np.array([[-2],[-1]])
print(V)
print('\n')
print(V/lag.Magnitude(V))

[[-2]
 [-1]]


[[-0.89442719]
 [-0.4472136 ]]


### Example

Let's try another example with a $3\times 3$ matrix.

$$
\begin{equation}
B = \left[ \begin{array}{rrr} 1 & 2 & 0  \\ 2 & -1 & 4 \\  0 & 3 & 1\end{array}\right]
\end{equation}
$$

In [31]:
B = np.array([[1,2, 0],[2,-1,4],[0,3,1]])
evalues, evectors = SLA.eig(B)

print(evalues)
print('\n')
print(evectors)

V_1 = evectors[:,0:1]
V_2 = evectors[:,1:2]
V_3 = evectors[:,2:3]

E_1 = evalues[0]
E_2 = evalues[1]
E_3 = evalues[2]

[-4.12310563+0.j  1.        +0.j  4.12310563+0.j]


[[ 3.19250163e-01  8.94427191e-01  4.19279023e-01]
 [-8.17776152e-01 -1.03817864e-16  6.54726338e-01]
 [ 4.78875244e-01 -4.47213595e-01  6.28918535e-01]]


We don't have the exact eigenvalues, but we can check that $BV_i - \lambda_iV_i = 0$ for $i=1, 2, 3$, allowing of for small precision error.  

In [38]:
print(B@V_1-E_1*V_1)

[[-4.44089210e-16+0.j]
 [ 2.22044605e-15+0.j]
 [-1.11022302e-15+0.j]]


Instead of $BV_i - \lambda_iV_i = 0$ for each $i$, we can package the calculations into a single matrix multiplication.  If $S$ is the matrix with columns $V_i$, then $BS$ is the matrix with columns $BV_i$.  This matrix should be compared to the matrix that has $\lambda_iV_i$ as its columns.  To construct this matrix we use a diagonal matrix $D$, that has the $\lambda_i$ as its diagonal entries.  The matrix product $SD$ will then have columns $\lambda_iV_i$.

$$
\begin{equation}
SD = \left[ \begin{array}{c|c|c} & & \\ V_1 & V_2 & V_3 \\ & & \end{array} \right]
\left[ \begin{array}{rrr} \lambda_1 & 0 & 0  \\ 0 & \lambda_2 & 0 \\  0 & 0 & \lambda_3 \end{array}\right]= 
\left[ \begin{array}{c|c|c} & & \\ \lambda_1V_1 & \lambda_2V_2 & \lambda_3V_3 \\ & & \end{array}\right]
\end{equation}
$$

We can now simply check that $BS-SD = 0$.

In [37]:
S = evectors

#  Since the eigenvalues are complex, it is best to use an array of complex numbers for D
D = np.zeros((3,3),dtype='complex128')
for i in range(3):
    D[i,i] = evalues[i]

print(D)
print('\n')
print(B@S-S@D)

[[-4.12310563+0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  1.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  4.12310563+0.j]]


[[-4.44089210e-16+0.j -6.66133815e-16+0.j  4.44089210e-16+0.j]
 [ 2.22044605e-15+0.j  5.47907073e-16+0.j  0.00000000e+00+0.j]
 [-1.11022302e-15+0.j -1.11022302e-16+0.j  0.00000000e+00+0.j]]


### Diagonal Factorization

The calculation that we used to verify the eigenvalues an eigenvectors is also very useful to construct another important matrix factorization.  Suppose that $A$ is an $n\times n$ matrix, $D$ is a diagonal matrix with the eigenvalues of $A$ along its diagonal, and $S$ is the $n\times n$ matrix with the eigenvectors of $A$ as its columns.  We have just seen that $AS=SD$.  If $S$ is invertible, we may also write $A=SDS^{-1}$, which is known as the **diagonalization** of $A$.

The diagonalization of $A$ is important because it provides us with a complete description of the action of $A$ in terms of its eigenvectors.  Consider an arbitrary vector $X$, and the product $AX$, computed by using the three factors in the diagonization.

- $S^{-1}X$ computes the coordinates of $X$ in terms of the eigenvectors of $A$.
- Multiplication by $D$ then simply scales each coordinate by the corresponding eigenvalue.
- Multiplication by $S$ gives the results with respect to the standard basis.